In [1]:
import os
import sys
from pprint import pprint
sys.path.append('.')

import pandas as pd
import numpy as np
from tqdm import tqdm
import optuna
from sklearn import metrics
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim

from models.gat.gat_pytorch import GAT
from models.gat import params as gat_params
from utils.utils import *
from runners import tools

In [25]:
# torch.__version__
import torch_geometric
torch_geometric.__version__

'1.5.0'

In [2]:
os.chdir('/home/lyz/co-phase-separation/PSGAT/')
DATA_ROOT = './data'

In [3]:
def dim_reduction_cor(X, y, k=20):
    cors = np.zeros((X.shape[1]))
    
    # calculate the correlation with y for each feature
    for i in range(X.shape[1]):
        cor = np.corrcoef(X[:, i], y)[0, 1]
        if not np.isnan(cor):
            cors[i] = cor
    
    features = np.zeros_like(cors).astype(bool)
    features[np.argsort(-cors)[:k]] = True
    
    return features, cors

In [4]:
def data(
    ppi, # ['integrate', 'biogrid-all', 'biogrid-htp']
    seqemb=False, # ProSE sequence embedding
    expr=False, # Gene expression
    subloc=False, # Sublocalization
    gomf=False, # GO molecular function
    comp=False, # GO cellular component
    weights=False,
    weights_thr=200, # edge weight threshold
    max_feat_dim=150, # k in feature selection
    seed=0 # random state
):
    # edges and edge_weights
    ppi_path = os.path.join(
        DATA_ROOT,           
        f'PPIN/{ppi.upper()}.csv'
    )
    edges = pd.read_csv(ppi_path)
    edge_weights = None
    
    if (ppi in ['string']) and (weights==True):      
        key = 'combined_score'
        edges = edges[edges.loc[:, key] > weights_thr].reset_index()
        edge_weights = edges['combined_score'] / 1000
        print(f'Filtered {ppi} network with thresh:', weights_thr)
    
    edges = edges[['A', 'B']].copy()
    edges = edges.dropna()
    edges, index = edges.values, edges.index.values
    ppi_genes = np.union1d(edges[:, 0], edges[:, 1])
    if edge_weights is not None:
        edge_weights = edge_weights.iloc[index].values
    
    # labels
    label_path = os.path.join(
        DATA_ROOT,
        f'Label/labels.csv'
    )
    labels = pd.read_csv(label_path).set_index('Gene')

    ## filter labels not in the PPI network
    print('Number of labels before filtering:', len(labels))
    labels = labels.loc[np.intersect1d(labels.index, ppi_genes)].copy()
    print('Number of labels after filtering:', len(labels))
    
    """正负样本比例：1:1"""
    ratio = 1
    pos_samples = labels[labels['label']==1]
    neg_samples = labels[labels['label']==0].sample(n=pos_samples.shape[0] * ratio, random_state=seed)
    labels = pd.concat([pos_samples, neg_samples]).sort_index()
    
    ## train and test split
    train_ds, test_ds = train_test_split(
        labels,
        test_size=.2,
        random_state=seed,
        stratify=labels
    )
    
    genes = np.union1d(labels.index, ppi_genes)
    print('Total number of genes:', len(genes))
    
    # node attributes
    X = np.zeros((len(genes), 0))
    X = pd.DataFrame(X, index=genes)
    
    ## ProSE sequence embedding
    if seqemb:
        seqemb_path = os.path.join(
            DATA_ROOT,           
            f'NodeFeat/SeqEmb/seqemb_80d.pkl'
        )
        seqemb_feat = pd.read_pickle(seqemb_path).set_index('entry')
        columns = [f'seqemb_{i}' for i in range(seqemb_feat.shape[1])]
        seqemb_feat.columns = columns
        X = X.join(seqemb_feat, how='left')
        print('Sequence Embedding dataset shape:', seqemb_feat.shape)
    
    ## Gene expression
    
    X = X.fillna(0)
    
    N = len(X)
    mapping = dict(zip(genes, range(N)))
    
    # preprocessing
    ## remove self-loops
    mask = edges[:, 0] != edges[:, 1]
    edges = edges[mask]
    if edge_weights is not None:
        edge_weights = edge_weights[mask]
    
    ## remove duplicated connections
    df = pd.DataFrame(edges, columns=['A', 'B'])
    df[0] = np.sort(df[['A', 'B']].values).sum(axis=1)
    df = df.drop_duplicates(subset=0)
    edges, index = df.iloc[:, :2].values, df.index.values
    if edge_weights is not None:
        edge_weights = edge_weights[index]
        edge_weights = torch.tensor(edge_weights, dtype=torch.float32)
    
    edge_index = np.vectorize(mapping.__getitem__)(edges)
    
    ## node attribute matrix X
    degrees = np.zeros((N, 1))
    nodes, counts = np.unique(edge_index, return_counts=True)
    degrees[nodes, 0] = counts
    
    if X is None or not X.shape[1]:
        X = np.random.random((N, 80))

    if X.shape[1] < 50:
        X = np.concatenate([X, np.random.random((N, 50))], axis=1)
        
    # X = np.concatenate([X, degrees.reshape((-1, 1))], 1) # concat degree vector
    X = X.to_numpy()
    X = (X - X.mean(0, keepdims=True)) / (X.std(0, keepdims=True) + 1e-8) # normalization
    
    ## train and val split
    train, val = train_test_split(
        train_ds,
        test_size=.05,
        random_state=seed,
        stratify=train_ds
    )
    
    train_idx = [mapping[t] for t in train.index]
    val_idx = [mapping[v] for v in val.index]
    test_idx = [mapping[v] for v in test_ds.index]
    
    # feature selection
    red_idx = np.concatenate([train_idx, test_idx, val_idx], axis=0)
    red_y = np.concatenate([train.label, test_ds.label, val.label], axis=0)
    feats, cors = dim_reduction_cor(
        X[red_idx], 
        red_y.astype(np.float32), 
        k=max_feat_dim
    )
    X = X[:, feats]
    
    # Torch
    edge_index = torch.from_numpy(edge_index.T)
    edge_index = edge_index.to(torch.long).contiguous()
    
    X = torch.from_numpy(X).to(torch.float32)
    train_y = torch.tensor(train.label.astype(int), dtype=torch.float32)
    val_y = torch.tensor(val.label.astype(int), dtype=torch.float32)
    test_y = torch.tensor(test_ds.label.astype(int), dtype=torch.float32)
    
    print(f'\nNumber of edges in graph: {len(edges)}')
    print(f'Number of nodes in graph: {len(X)}')
    print(f'Shape of node features: {X.shape[0], X.shape[1]}\n')
    print('Using Edge Weights' if edge_weights is not None else 'Not using edge weights')
    
    return (edge_index, edge_weights), X, (train_idx, train_y), (val_idx, val_y), \
            (test_idx, test_y), genes

In [5]:
# (edge_index, edge_weights), X, (train_idx, train_y), (val_idx, val_y), (test_idx, test_y), genes = data(ppi='biogrid-all', seqemb=True, weights_thr=250, weights=False)

In [5]:
gat_0 = {
    'lr': 0.005,
    'weight_decay': 5e-4,
    'h_feats': [8, 1],
    'heads': [8, 1],
    'dropout': 0.6,
    'negative_slope': 0.2}

In [6]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
# Loss class
class Loss():
    def __init__(self, y, idx):
        self.y = y
        idx = np.array(idx)

        self.y_pos = y[y == 1]
        self.y_neg = y[y == 0]

        self.pos = idx[y.cpu() == 1]
        self.neg = idx[y.cpu() == 0]

    def __call__(self, out):
        loss_p = F.binary_cross_entropy_with_logits(
            out[self.pos].squeeze(), self.y_pos)
        loss_n = F.binary_cross_entropy_with_logits(
            out[self.neg].squeeze(), self.y_neg)
        loss = loss_p + loss_n
        return loss

# AUC calculation
def evalAUC(model, X, A, y, mask, logits=None):
    assert(model is not None or logits is not None)
    if model is not None:
        model.eval()
        with torch.no_grad():
            logits = model(X, A)
            logits = logits[mask]
    probs = torch.sigmoid(logits)
    probs = probs.cpu().numpy()
    y = y.cpu().numpy()
    auc = metrics.roc_auc_score(y, probs)
    fpr, tpr, _ = metrics.roc_curve(y, probs)
    return auc, fpr, tpr

# Model training
def train(
    params,
    X, A,
    edge_weights,
    train_y, train_idx,
    val_y, val_idx,
    savepath=''
):
    epochs = 1000
    
    model = GAT(in_feats=X.shape[1], **params)
    model.to(DEVICE)
    X = X.to(DEVICE)
    A = A.to(DEVICE)
    train_y = train_y.to(DEVICE)
    val_y = val_y.to(DEVICE)
    if edge_weights is not None:
        edge_weights = edge_weights.to(DEVICE)
    
    optimizer = optim.Adam(
        model.parameters(),
        lr=params['lr'],
        weight_decay=params['weight_decay']
    )
    loss_fnc = tools.Loss(train_y, train_idx)
    val_loss_fnc = tools.Loss(val_y, val_idx)
    
    iterable = tqdm(range(epochs))
    for i in iterable:
        model.train()
        logits = model(X, A, edge_weights=edge_weights)
        
        optimizer.zero_grad()
        loss = loss_fnc(logits)
        loss.backward()
        optimizer.step()
        
        logits = logits.detach()
        val_loss = val_loss_fnc(logits)
        train_auc, _, _ = evalAUC(None, 0, 0, train_y, 0, logits[train_idx])
        val_auc, _, _ = evalAUC(None, 0, 0, val_y, 0, logits[val_idx])
        
        tqdm.set_description(
            iterable,
            desc='Loss: %.4f; Val Loss %.4f; Train AUC %.4f. Validation AUC: %.4f' % (loss, val_loss, train_auc, val_auc)
        )
    
    score, fpr, tpr = evalAUC(model, X, A, val_y, val_idx)
    auc_dict = {
        'auc': score,
        'fpr': fpr,
        'tpr': tpr
    }
    print(f'Val AUC: {score}')
    
    return model, auc_dict

# Model testing
def test(model, X, A, test_ds=None):
    model.to(DEVICE).eval()
    X = X.to(DEVICE)
    A = A.to(DEVICE)
    
    with torch.no_grad():
        logits = model(X, A)
    probs = torch.sigmoid(logits)
    probs = probs.cpu().numpy()
    
    if test_ds is not None:
        test_idx, test_y = test_ds
        test_y = test_y.cpu().numpy()
        # roc curve and auc
        auc = metrics.roc_auc_score(test_y, probs[test_idx])
        fpr, tpr, _ = metrics.roc_curve(test_y, probs[test_idx])
        auc_dict = {
            'auc': auc,
            'fpr': fpr,
            'tpr': tpr
        }
        # prc curve and auprc
        precision, recall, _ = metrics.precision_recall_curve(test_y, probs[test_idx], pos_label=1)
        auprc = metrics.auc(recall, precision)
        auprc_dict = {
            'auprc': auprc,
            'pre': precision,
            'rec': recall
        }
        return probs, auc_dict, auprc_dict
    return probs

In [8]:
N = 10
models, preds, val_aucs, test_aucs, test_auprcs = (dict() for i in range(5))

In [9]:
for i in range(N):
    (edge_index, edge_weights), X, (train_idx, train_y), (val_idx, val_y), (test_idx, test_y), genes = \
                                                    data(ppi='randomgraph', 
                                                         seqemb=True,
                                                         weights_thr=0, 
                                                         weights=False, 
                                                         seed=i)
    model, val_auc = train(gat_0, X, edge_index, edge_weights, train_y, train_idx, val_y, val_idx)
    pred, test_auc, test_auprc = test(model, X, edge_index, (test_idx, test_y))
#     break
    models[i] = model
    preds[i] = pred
    val_aucs[i], test_aucs[i] = val_auc, test_auc
    test_auprcs[i] = test_auprc
    print('Model_{}, AUROC:{}, AUPRC:{}\n'.format(i, test_auc['auc'], test_auprc['auprc']))

Number of labels before filtering: 20398
Number of labels after filtering: 15939
Total number of genes: 15939
Sequence Embedding dataset shape: (20398, 80)

Number of edges in graph: 239188
Number of nodes in graph: 15939
Shape of node features: (15939, 80)

Not using edge weights


Loss: 1.1179; Val Loss 1.6313; Train AUC 0.7821. Validation AUC: 0.5249: 100%|██████████| 1000/1000 [00:25<00:00, 38.77it/s]


Val AUC: 0.5318877551020407
Model_0, AUROC:0.610837948346359, AUPRC:0.5889589404417019

Number of labels before filtering: 20398
Number of labels after filtering: 15939
Total number of genes: 15939
Sequence Embedding dataset shape: (20398, 80)


Loss: 1.4807; Val Loss 1.4070; Train AUC 0.5009. Validation AUC: 0.5517:   0%|          | 4/1000 [00:00<00:27, 36.52it/s]


Number of edges in graph: 239188
Number of nodes in graph: 15939
Shape of node features: (15939, 80)

Not using edge weights


...

Val AUC: 0.4936224489795919
Model_8, AUROC:0.5067543087831893, AUPRC:0.5046703749901394

Number of labels before filtering: 20398
Number of labels after filtering: 15939
Total number of genes: 15939
Sequence Embedding dataset shape: (20398, 80)


Loss: 1.5603; Val Loss 1.6245; Train AUC 0.4935. Validation AUC: 0.4305:   0%|          | 4/1000 [00:00<00:27, 35.93it/s]


Number of edges in graph: 239188
Number of nodes in graph: 15939
Shape of node features: (15939, 80)

Not using edge weights


Loss: 1.2499; Val Loss 1.6707; Train AUC 0.6951. Validation AUC: 0.5842: 100%|██████████| 1000/1000 [00:25<00:00, 39.76it/s]


Val AUC: 0.5012755102040816
Model_9, AUROC:0.5496092334765281, AUPRC:0.5470051880927262



In [10]:
# average AUC for validation set
avgValAUC = np.mean([v['auc'] for v in val_aucs.values()])
# average AUC for test set
avgTestAUC = np.mean([v['auc'] for v in test_aucs.values()])
# average AUPRC for test set
avgTestAUPRC = np.mean([v['auprc'] for v in test_auprcs.values()])

In [11]:
print('Average AUC for val: {:.2f}, test: {:.2f}, AUPRC for test: {:.2f}'.format(avgValAUC, avgTestAUC, avgTestAUPRC))

Average AUC for val: 0.56, test: 0.56, AUPRC for test: 0.55


In [12]:
# mkdir
SAVE_ROOT = './saves/RANDGRAPH_ProSE80d_pos1neg1/'
if not os.path.exists(SAVE_ROOT):
    os.mkdir(SAVE_ROOT)
    os.mkdir(os.path.join(SAVE_ROOT, f'embeddings/'))
    os.mkdir(os.path.join(SAVE_ROOT, f'pairwise_cosine/'))
    os.mkdir(os.path.join(SAVE_ROOT, f'edge_cosine/'))
    os.mkdir(os.path.join(SAVE_ROOT, f'models/'))
    
# save models
for idx, model in tqdm(models.items()):
    torch.save(model, os.path.join(SAVE_ROOT, f'models/model_{idx}.pt'))
    torch.save(model.featuremap1.cpu(), os.path.join(SAVE_ROOT, f'embeddings/model_{idx}.pt'))

# save results
import pickle
dict1 = {
    'preds': preds,
    'val_aucs': val_aucs,
    'test_aucs': test_aucs,
    'test_auprcs': test_auprcs,
    'genes': genes
}
for key, val in dict1.items():
    with open(os.path.join(SAVE_ROOT, f'{key}.pkl'), 'wb') as f:
        pickle.dump(val, f)

# with open('saved_dictionary.pkl', 'rb') as f:
#     loaded_dict = pickle.load(f)

100%|██████████| 10/10 [00:00<00:00, 44.03it/s]


In [13]:
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

def save_models(ppi, model):
    ppi_path = os.path.join(
        DATA_ROOT,           
        f'PPIN/{ppi.upper()}.csv'
    )
    edges = pd.read_csv(ppi_path)

    featuremap = model.featuremap1.cpu()
    featuremap_cosine = pd.DataFrame(cosine_similarity(featuremap))
    featuremap_cosine.columns, featuremap_cosine.index = genes, genes

    cosim_list = list()
    for _, x in edges.iterrows():
        cosim = featuremap_cosine[x['A']][x['B']]
        cosim_list.append(cosim)
    edges['cosim'] = cosim_list
    edges.loc[edges['cosim'] < 0, 'cosim'] = 0
    edges.loc[edges['cosim'] > 1, 'cosim'] = 1
    
    return featuremap_cosine, edges

In [14]:
# save node embeddings
for idx, model in tqdm(models.items()):
    featuremap_cosine, featuremap_edge = save_models('integrate', model)
    featuremap_cosine.to_pickle(os.path.join(SAVE_ROOT, f'pairwise_cosine/model_{idx}.pkl'))
    featuremap_edge.to_pickle(os.path.join(SAVE_ROOT, f'edge_cosine/model_{idx}.pkl'))

100%|██████████| 10/10 [02:22<00:00, 14.28s/it]
